In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, sys

nb_path = '/content/drive/MyDrive/Colab Notebooks/site-packages'
sys.path.insert(0, nb_path)  # or append(nb_path)

In [3]:
# Standard library imports
import numpy as np

from gluonts.dataset.common import ListDataset
from gluonts.model.estimator import GluonEstimator
from gluonts.model.predictor import Predictor
from gluonts.trainer import Trainer

import mxnet as mx
from mxnet import gluon
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: gluonts.trainer is deprecated. Use gluonts.mx.trainer instead.
  import sys


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

In [5]:
mx.random.seed(0)
np.random.seed(0)

## **deepAR**

In [67]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ppikppik/outfree_gs_boo.csv", encoding='EUC-KR')

In [68]:
year_start = 1985
month = 12
day = 1
cnt_year = 0
for i in df.index :
  df.loc[i,'회계년도'] = pd.Timestamp(year=year_start+cnt_year, month=12, day=1)
  cnt_year = cnt_year + 1

In [69]:
temp = df.set_index(df['회계년도'])
temp[{'유동자산(요약)', '매출총이익(요약)', '매출액(요약)', '당기순이익(요약)', '영업이익(요약)'}]

,매출총이익(요약),유동자산(요약),당기순이익(요약),영업이익(요약),매출액(요약)
회계년도,,,,,
1985-12-01,8823.0,93153,2444.0,2576.0,105579
1986-12-01,14875.0,110459,3818.0,7120.0,152111
1987-12-01,20333.0,129208,4748.0,11035.0,240839
1988-12-01,23093.0,140120,3135.0,11398.0,277801
1989-12-01,20759.0,157434,1587.0,9807.0,266215
1990-12-01,29451.0,200084,1637.0,19810.0,280779
1991-12-01,22797.0,188209,3002.0,13032.0,285351
1992-12-01,32298.0,283401,10295.0,19108.0,324475
1993-12-01,44634.0,360086,14174.0,28015.0,427284


In [70]:
def find_upper_lower_bound(df,col):
    temp = sorted(df[col])
    q1, q3 = np.percentile(temp, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    return lower_bound, upper_bound

def range_check_correct(df,col):
    cor_range = find_upper_lower_bound(df,col)
    print(cor_range)
    for i in df.index:
        if df.loc[i][col] < (float)(cor_range[0]) :
            df[col][i] = (float)(cor_range[0])
        if df.loc[i][col] > (float)(cor_range[1]) :
            df[col][i] = (float)(cor_range[1])
    return df

In [71]:
temp = range_check_correct(temp,'영업이익(요약)')

(-294324.625, 642062.375)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [72]:
temp

,회사명,연도,거래소코드,회계년도,자산총계(요약),부채총계(요약),자본총계(요약),매출액(요약),매출원가(요약),매출총이익(요약),판매비와 관리비(요약),영업이익(요약),당기순이익(요약),유동자산(요약),유동부채(요약),비유동부채(요약),자본금(요약),비유동자산(요약),총자산수익률(ROA),자기자본수익률(ROE),영업이익률,당기순이익률,유동비율,자기자본비율,부채비용,비유동비율,비유동장기적합률
회계년도,,,,,,,,,,,,,,,,,,,,,,,,,,,
1985-12-01,지에스건설(주),1981,6360,1985-12-01 00:00:00,128425,113446,14980,105579,96756,8823.0,6248.0,2576.000,2444.0,93153,104252,9194,6100,35273,0.019031,0.163151,0.024399,0.023149,0.893537,0.116643,18.597705,0.260653,2.306329
1986-12-01,지에스건설(주),1982,6360,1986-12-01 00:00:00,151771,128452,23318,152111,137236,14875.0,7754.0,7120.000,3818.0,110459,97501,30951,12200,41312,0.025156,0.163736,0.046808,0.025100,1.132901,0.153640,10.528852,0.749201,0.957382
1987-12-01,지에스건설(주),1983,6360,1987-12-01 00:00:00,166438,139283,27154,240839,220506,20333.0,9298.0,11035.000,4748.0,129208,120003,19280,12200,37230,0.028527,0.174855,0.045819,0.019714,1.076706,0.163149,11.416639,0.517862,1.182656
1988-12-01,지에스건설(주),1984,6360,1988-12-01 00:00:00,182495,153671,28824,277801,254708,23093.0,11695.0,11398.000,3135.0,140120,123079,30592,12200,42375,0.017179,0.108764,0.041029,0.011285,1.138456,0.157944,12.595984,0.721935,0.990255
1989-12-01,지에스건설(주),1985,6360,1989-12-01 00:00:00,206772,176681,30091,266215,245456,20759.0,10952.0,9807.000,1587.0,157434,139200,37481,12200,49338,0.007675,0.052740,0.036839,0.005961,1.130991,0.145527,14.482049,0.759678,0.993096
1990-12-01,지에스건설(주),1986,6360,1990-12-01 00:00:00,249015,219908,29108,280779,251328,29451.0,9641.0,19810.000,1637.0,200084,163427,56481,12200,48931,0.006574,0.056239,0.070554,0.005830,1.224302,0.116892,18.025246,1.154299,0.712439
1991-12-01,지에스건설(주),1987,6360,1991-12-01 00:00:00,223617,181387,42230,285351,262554,22797.0,9765.0,13032.000,3002.0,188209,115760,65627,24400,35409,0.013425,0.071087,0.045670,0.010520,1.625855,0.188850,7.433893,1.853399,0.393315
1992-12-01,지에스건설(주),1988,6360,1992-12-01 00:00:00,347019,248614,98406,324475,292177,32298.0,13190.0,19108.000,10295.0,283401,173079,75535,40000,63619,0.029667,0.104618,0.058889,0.031728,1.637408,0.283574,6.215350,1.187303,0.550647
1993-12-01,지에스건설(주),1989,6360,1993-12-01 00:00:00,487910,379863,108047,427284,382651,44634.0,16619.0,28015.000,14174.0,360086,237840,142023,40000,127824,0.029050,0.131184,0.065565,0.033172,1.513984,0.221449,9.496575,1.111082,0.702241


In [73]:
prediction_window = 1

In [74]:
train_ds = ListDataset(
    [{"start": temp.index[0], "target": temp['영업이익(요약)'].values[:-prediction_window] ,
      "feat_dynamic_real": [temp['유동자산(요약)'].values[:-prediction_window],temp['매출총이익(요약)'].values[:-prediction_window],temp['매출액(요약)'].values[:-prediction_window],temp['당기순이익(요약)'].values[:-prediction_window]]}],
                        freq = "365D"
)

In [75]:
freq = "365D"

estimator = DeepAREstimator(prediction_length = prediction_window, use_feat_dynamic_real= True , freq=freq, cell_type = 'lstm', 
                           trainer=Trainer(epochs=200, ctx = mx.context.gpu(),))

In [76]:
predictor = estimator.train(training_data = train_ds)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:00<00:00, 75.74it/s, epoch=177/200, avg_epoch_loss=7.62]


In [77]:
from gluonts.evaluation.backtest import make_evaluation_predictions

test_data = ListDataset(
    [{"start": temp.index[0], "target": temp['영업이익(요약)'].values,
      "feat_dynamic_real": [temp['유동자산(요약)'].values,temp['매출총이익(요약)'].values,temp['매출액(요약)'].values,temp['당기순이익(요약)'].values]}],
    freq = "365D"
)

forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_data,  # test dataset
    predictor=predictor ,  # predictor
    num_samples=1,  # number of sample paths we want for evaluation
)

In [78]:
answer_data = ListDataset(
    [{"start": temp.index[0], "target": temp['영업이익(요약)'].values}],
    freq = "365D"
)

In [79]:
forecasts = list(forecast_it)
tss = list(ts_it)

In [80]:
train_entry = next(iter(forecasts))
train_entry

gluonts.model.forecast.SampleForecast(freq="365D", info=None, item_id=None, samples=numpy.array([[828457.5]], numpy.dtype("float32")), start_date=pandas.Timestamp("2022-12-19 00:00:00", freq="365D"))

In [81]:
train_entry.samples

array([[828457.5]], dtype=float32)

In [82]:
predict = train_entry.samples

In [83]:
predict[0][0]

828457.5

In [84]:
answer = temp['영업이익(요약)'].values[-prediction_window:]

In [85]:
answer[0]

642062.375

In [86]:
RMSE = mean_squared_error(answer,predict[0])**0.5

In [87]:
RMSE

186395.125